# Calculation of image quality

Will pull the ImageId from the MongoDB to then grab the associated image from the image server. Then calculate various quality metrics. Finally build a dataframe with the Image ID, and quality scores. Then push back the new info to MongoDB.

`!ssh -f -N -L 27017:irlinv-tellus:27017 irlinv-tellus`

In [ ]:
import io, os, sys
import requests
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from collections import Counter
from ipywidgets import IntProgress
from IPython.display import display

import skimage as ski
import cv2
import torch
import piq
#import nvidia_smi

sys.path.append('..')

# local library of functions to connect to image server
#import calcimetry.use_server as server
from calcimetry.mongo_api import MongoInfo, MongoAPI
from calcimetry.calcimetry_api import CalcimetryAPI

# parameters where the database is stored, can obviously be distant.
sandbox = False
if os.getcwd() == '/work/home/ai.calcimetry/notebooks':
    print('On the sandbox')
    sandbox = True
    HOST='irlinv-tellus'
    PORT=27017
else:
    # ssh -f -N -L 27017:irlinv-tellus:27017 irlinv-tellus
    HOST='localhost'
    PORT=27017   

In [ ]:
#img_path = server.init()
mongo_info = MongoInfo(host=HOST, port=PORT)

In [ ]:
with MongoAPI(mongo_info=mongo_info) as mongo_api:
    doc = mongo_api.db['images'].find()
    df = pd.DataFrame(list(doc))
df

### The variance of the Laplacian can be a measure of the sharpness of the image, or the focus

In [ ]:
def variance_of_laplacian(image):
    # compute the Laplacian of the image and then return the focus
    # measure, which is simply the variance of the Laplacian
    return cv2.Laplacian(image, cv2.CV_64F).var()

### Magnitude of the gradient to get sharpness of edges

In [ ]:
def gradient_magnitude(image):
    #Get magnitude of gradient for given image
    ddepth = cv2.CV_64F
    dx = cv2.Sobel(image, ddepth, 1, 0)
    dy = cv2.Sobel(image, ddepth, 0, 1)
    mag = cv2.magnitude(dx, dy)
    return mag

## Colour analysis
https://towardsdatascience.com/building-an-image-color-analyzer-using-python-12de6b0acf74

* First, we are using k-Means to cluster the top colors. Inside the function we are passing the value of how many clusters do we want to divide. Here is the documentation for K-Means clustering. After clustering we predict the colors that weigh the most — meaning getting the most area on the image.
* Secondly, we are calling the Counter function. Counter creates a container to the elements as dictionary keys, and their volume is store as dictionary values. If you are not familiar with dictionaries, they store data in key: value pairs. They are like function, and when you pass in the “key,” you can “value” as a return. Then we are ordering the colors according to the keys.

In [ ]:
def prep_image(raw_img):
    modified_img = cv2.resize(raw_img, (900, 600), interpolation = cv2.INTER_AREA)
    modified_img = modified_img.reshape(modified_img.shape[0]*modified_img.shape[1], 3)
    return modified_img

def color_analysis(img):
    clf = KMeans(n_clusters = 5)  # 5 top colours
    color_labels = clf.fit_predict(img)
    center_colors = clf.cluster_centers_
    counts = Counter(color_labels)
    ordered_colors = [center_colors[i] for i in counts.keys()]
    return ordered_colors

### From the dataframe caclulate some metrics

Add some Facebook metrics too: `piq` PyTorch Image Quality
* https://github.com/photosynthesis-team/piq/blob/master/examples/image_metrics.py

In [ ]:
Resolution = []
Focus = []
GradientMax = []
GradientSTD = []
Colour1 = []
Colour2 = []
Colour3 = []
Colour4 = []
Colour5 = []
BRISQUE_i = []

missing_images = []

f = IntProgress(min=0, max=len(df)) # instantiate the bar
display(f) # display the bar

for ImageId in df['ImageId']:
    
    with CalcimetryAPI(mongo_info=mongo_info) as calcimetry_api:
        img = calcimetry_api.read_image(ImageId)
        
        if img is not None:

            # focus metric
            gray = cv2.cvtColor(np.asarray(img.jpg), cv2.COLOR_BGR2GRAY)
            Focus.append(variance_of_laplacian(gray))

            # Gradient metric
            GradientMax.append(np.max(gradient_magnitude(gray)[:]))
            GradientSTD.append(np.std(gradient_magnitude(gray)[:]))

            # Top five colours
            try:
                colours = color_analysis(prep_image(np.asarray(img.jpg)))
                Colour1.append(colours[0])
                Colour2.append(colours[1])
                Colour3.append(colours[2])
                Colour4.append(colours[3])
                Colour5.append(colours[4])
            except:
                Colour1.append(np.nan)
                Colour2.append(np.nan)
                Colour3.append(np.nan)
                Colour4.append(np.nan)
                Colour5.append(np.nan)

            if sandbox == True:
            # pytorch image quality, use try, except loop to keep going if image is too large or does not conform
                try:
                    x = torch.tensor(np.asarray(img.jpg)).permute(2, 0, 1)[None, ...] / 255.
                    if torch.cuda.is_available():
                        # Move to GPU to make computaions faster
                        # print(torch.cuda.is_available())
                        x = x.cuda()
                    brisque_index: torch.Tensor = piq.brisque(x, data_range=1., reduction='none')

                    BRISQUE_i.append(brisque_index.item())
                except Exception as e:
                    print(f'Error in PyTorch with image {ImageId}\n')
                    print(e)
                    BRISQUE_i.append(np.nan)

                if torch.cuda.is_available():
                    torch.cuda.empty_cache()
                del x

        else:  # not an image
            print(f'{fileName} is not a jpg')
            missing_images.append(fileName)
            Focus.append(np.nan)
            GradientMax.append(np.nan)
            GradientSTD.append(np.nan)
            Colour1.append(np.nan)
            Colour2.append(np.nan)
            Colour3.append(np.nan)
            Colour4.append(np.nan)
            Colour5.append(np.nan)
            BRISQUE_i.append(np.nan)
            
        f.value += 1

In [ ]:
# resolution
dpxdx = (df['px1'] - df['px0']) / (df['Cote1'] - df['Cote0'])

In [ ]:
plt.imshow(img)
plt.show()